<a href="https://colab.research.google.com/github/cheolhakja/fine-dust-prediction/blob/main/graduation_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pylab as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import datasets

print(1*1)

# Load the diabetes dataset.
# diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Select only one feature (BMI) and make into a 2-D array. The index of BMI feature is 2.
# diabetes_X_new = diabetes_X[:, np.newaxis, 2]

# Separate training data from test data.
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(diabetes_X_new, diabetes_y, test_size=0.1, random_state=0)



1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/graduation_project/station_info.csv')
station_names = df['stationName']

print(station_names)

0        중구
1      한강대로
2       종로구
3      청계천로
4        종로
       ... 
162     본오동
163     원곡동
164    부곡동1
165     대부동
166     호수동
Name: stationName, Length: 167, dtype: object


In [ ]:
# 대기 정보 데이터 수집

import requests
import certifi
import ssl
import time
from requests.adapters import HTTPAdapter
from urllib3.poolmanager import PoolManager
from google.colab import drive

class SSLAdapter(HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, *args, **kwargs):
        kwargs['ssl_context'] = self.ssl_context
        return super().init_poolmanager(*args, **kwargs)

# 커스텀 context 설정
ctx = ssl.create_default_context()
ctx.load_verify_locations(certifi.where())
ctx.set_ciphers('DEFAULT:@SECLEVEL=1')  # 핵심 설정: 보안 레벨 낮추기

session = requests.Session()
session.mount("https://", SSLAdapter(ssl_context=ctx))

drive.mount('/content/drive')
api_key = '/content/drive/MyDrive/graduation_project/airkorea_api_key_2.txt'

all_items = []

url = "https://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty"


#################################################################
# 일일 트래픽 한도로 인해, 여러번 나누어 api 호출하기
#################################################################
all_stations = station_names.tolist()

test_stations_1 = station_names[:5].tolist()
target_stations = test_stations_1

test_stations_2 = station_names[5:45].tolist()
target_stations = test_stations_2

test_stations_3 = station_names[45:100].tolist()
target_stations = test_stations_3

test_stations_4 = station_names[132:].tolist()
target_stations = test_stations_4

test_stations_5 = ["용문면"]
target_stations = test_stations_5
#################################################################

for station in target_stations:
    time.sleep(0.3)  # 측정소마다 0.3초 쉬어줌 (rate limit 방지)
    page = 1
    total_items = []

    # 첫 페이지 요청
    params = {
        'serviceKey': api_key,
        'returnType': 'json',
        'numOfRows': '1000',
        'pageNo': str(page),
        'stationName': station,
        'dataTerm': '3MONTH',
        'ver': '1.5'
    }

    try:
        res = session.get(url, params=params, timeout=10)
        data = res.json()

        if data['response']['header']['resultCode'] != '00':
            print(f"❌ {station} 1페이지 에러:", data['response']['header']['resultMsg'])
            continue

        body = data['response']['body']
        items = body.get('items', [])
        totalCount = int(body.get('totalCount', 0))

        if not items:
            print(f"⚠️ {station} 1페이지에 데이터 없음")
            continue

        for item in items:
            item['station'] = station
        total_items.extend(items)

        total_pages = (totalCount - 1) // 100 + 1

        # 나머지 페이지 반복
        for i in range(2, total_pages + 1):
            time.sleep(0.15)  # 페이지마다도 약간 쉬어줌
            params['pageNo'] = str(i)
            res = session.get(url, params=params, timeout=10)
            data = res.json()

            if data['response']['header']['resultCode'] != '00':
                print(f"❌ {station} {i}페이지 에러:", data['response']['header']['resultMsg'])
                break

            items = data['response']['body'].get('items', [])
            if not items:
                print(f"⚠️ {station} {i}페이지에 데이터 없음")
                break

            for item in items:
                item['station'] = station
            total_items.extend(items)

        print(f"✅ {station} 데이터 수집 완료 ({len(total_items)}개)")

    except Exception as e:
        print(f"❌ {station} {page}페이지 파싱 실패:", e)
        print("응답 원문:", res.text)

    all_items.extend(total_items)







⚠️ 용문면 4페이지에 데이터 없음
✅ 용문면 데이터 수집 완료 (2182개)


In [ ]:
# 구글 드라이브에 저장

import pandas as pd
import os
from datetime import datetime

os.makedirs('/content/drive/MyDrive/graduation_project', exist_ok=True)

df = pd.DataFrame(all_items)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = f'/content/drive/MyDrive/graduation_project/air_quality_data_{timestamp}.csv'

df.to_csv(save_path, index=False)

print("✅ 저장 완료:", save_path)

df.head()



✅ 저장 완료: /content/drive/MyDrive/graduation_project/air_quality_data_20250702_100842.csv


,pm25Grade1h,pm10Value24,so2Value,pm10Grade1h,o3Grade,pm10Value,pm25Flag,khaiGrade,pm25Value,no2Flag,...,no2Grade,pm25Value24,o3Flag,pm25Grade,so2Flag,coGrade,dataTime,pm10Grade,o3Value,station
0,1,18,0.0027,1,2,14,None,2,12,None,...,1,8,None,1,None,1,2025-07-02 18:00,1,0.0321,용문면
1,1,20,0.0027,1,2,15,None,2,5,None,...,1,7,None,1,None,1,2025-07-02 17:00,1,0.032,용문면
2,1,20,0.0028,1,2,14,None,2,6,None,...,1,7,None,1,None,1,2025-07-02 16:00,1,0.0304,용문면
3,1,-,0.0029,1,1,20,None,None,8,None,...,1,-,None,None,None,1,2025-07-02 15:00,None,0.0292,용문면
4,1,-,0.0033,1,2,21,None,None,7,None,...,1,-,None,None,None,1,2025-07-02 14:00,None,0.0362,용문면


In [ ]:
#csv에 저장된 측정소 갯수
print(len(df))         # 전체 row 수
print(df['stationName'].nunique())

76131
35


In [ ]:
#측정소 정보 조회(이름, 위도, 경도)

import requests
import certifi
import ssl
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.poolmanager import PoolManager
from google.colab import drive

# SSLAdapter 클래스 정의
class SSLAdapter(HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, *args, **kwargs):
        kwargs['ssl_context'] = self.ssl_context
        return super().init_poolmanager(*args, **kwargs)

# 보안레벨 낮춘 SSL 설정
ctx = ssl.create_default_context()
ctx.load_verify_locations(certifi.where())
ctx.set_ciphers('DEFAULT:@SECLEVEL=1')

# Session 생성
session = requests.Session()
session.mount("https://", SSLAdapter(ssl_context=ctx))

# ✅ URL 인코딩된 인증키 사용 (중요!)
drive.mount('/content/drive')
api_key = '/content/drive/MyDrive/graduation_project/airkorea_api_key_2.txt'
url = "https://apis.data.go.kr/B552584/MsrstnInfoInqireSvc/getMsrstnList"

station_info = []

# ✅ 서울/경기 각각 조회
for sido in ['서울', '경기']:
    params = {
        'serviceKey': api_key,
        'returnType': 'json',
        'addr': sido,
        'numOfRows': 1000
    }

    try:
        res = session.get(url, params=params, timeout=10)
        data = res.json()
        items = data['response']['body']['items']
        print(f"\n📍 {sido} 측정소 (일부):")
        for item in items[:5]:
            print(f" - {item['stationName']} ({item['addr']})")
        for item in items:
            station_info.append({
                'sido': sido,
                'stationName': item['stationName'],
                'addr': item['addr'],
                'dmX': item['dmX'],
                'dmY': item['dmY']
            })
    except Exception as e:
        print(f"❌ {sido} 파싱 실패:", e)
        print("응답 원문:", res.text)

# DataFrame으로 변환
df_station = pd.DataFrame(station_info)
df_station.to_csv("station_info.csv", index=False, encoding='utf-8-sig')
df_station.head()

save_path = "/content/drive/MyDrive/graduation_project/station_info.csv"
df_station.to_csv(save_path, index=False, encoding='utf-8-sig')



In [ ]:
### test



In [ ]:
# Linear Regression with sklearn
# regr = LinearRegression()
# regr.fit(X_train, y_train)

# Let's make predictions with test data.
# y_pred = regr.predict(X_test)

# # Compare the real data with our prediction.
# plt.scatter(X_test, y_test,  color='black')
# plt.plot(X_test,y_pred, color='blue', linewidth=3)
# plt.show()

In [ ]:
# Trained weights with sklearn
# regr.coef_, regr.intercept_

In [ ]:
# Training data for your Linear regression algorithm
# X_train = X_train[:,0]
# X_test = X_test[:,0]


# Todo: Implement the Gradient Descent Algorithm for Linear Regression


In [ ]:
# Train W and b using the training data only
# Use X_train and y_train only

# W = np.random.rand()        # Initialization of W
# b = np.random.rand()        # Initialization of b

# epochs = 25000              # number of epochs
# n = float(len(X_train))     # number of training samples
# lr = 0.1                    # learning rate

# train_loss = []

# for k in range(epochs):
# Implement the gradient descent here


In [ ]:
# print("Trained parameters")
# print("W:", W,", b:", b)

In [ ]:
# Checking for traing: Using training data
# use X_train and y_train

# y_pred = W*X_train + b
# plt.scatter(X_train, y_train,  color='black')
# plt.plot(X_train,y_pred, color='blue', linewidth=3)
# plt.title("Linear Regression Training Results")
# plt.show()

In [ ]:
# Prediction: Using only the test data
# use X_test and y test only

# y_pred = W*X_test + b
# plt.scatter(X_test, y_test,  color='black')
# plt.plot(X_test,y_pred, color='blue', linewidth=3)
# plt.title("Linear Regression Test")
# plt.show()

In [ ]:
# Display the loss at every epoch (sum of squares error on your training data)
# plt.plot(np.arange(epochs), train_loss,  color='black')
# plt.title("Loss vs epochs")
# plt.xlabel("epochs")
# plt.ylabel("Training Loss")
# plt.show()

In [ ]:
# Display the same, but only for last 10000 epochs

# plt.plot(np.arange(epochs)[15000:], train_loss[15000:],  color='black')
# plt.title("Loss vs last 10000 epochs")
# plt.xlabel("Last 10000 epochs")
# plt.ylabel("Training Loss")
# plt.show()


In [ ]:
#남은 일일 트래픽 횟수 조회

import requests
import ssl
import certifi
from datetime import datetime
from requests.adapters import HTTPAdapter
from urllib3.poolmanager import PoolManager
from google.colab import drive

# 🔧 SSL 어댑터 정의
class SSLAdapter(HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(self, *args, **kwargs):
        kwargs['ssl_context'] = self.ssl_context
        return super().init_poolmanager(*args, **kwargs)

# 🔧 SSL 세션 생성
ctx = ssl.create_default_context()
ctx.load_verify_locations(certifi.where())
ctx.set_ciphers('DEFAULT:@SECLEVEL=1')

session = requests.Session()
session.mount("https://", SSLAdapter(ssl_context=ctx))

# 📌 서비스 키
drive.mount('/content/drive')
api_key = '/content/drive/MyDrive/graduation_project/airkorea_api_key_2.txt'

# 📌 트래픽 조회 API
traffic_url = "https://apis.data.go.kr/B552584/UserSportSvc/getSvkceyDalyStats"

# 📌 오늘 날짜
today = datetime.now().strftime('%Y-%m-%d')

# ✅ 파라미터 설정
params = {
    'serviceKey': api_key,
    'searchDate': today,
    'returnType': 'json',
    'numOfRows': '100',
    'pageNo': '1'
}

# ✅ API 요청
try:
    res = session.get(traffic_url, params=params, timeout=10)
    data = res.json()

    # 응답 확인
    items = data['response']['body']['items']
    if len(items) == 0:
        print("❌ 응답에 트래픽 정보가 없음")
    else:
        use_cnt = int(items[0]['useCnt'])
        daily_limit = 1000  # 👉 네 계정 2개로 최대 트래픽 수 설정
        remaining = daily_limit - use_cnt

        print(f"📊 오늘 사용한 트래픽 수: {use_cnt}회")
        print(f"✅ 남은 트래픽 수: {remaining}회")

except Exception as e:
    print("❌ 트래픽 조회 실패:", e)
    print("응답 원문:", res.text)


❌ 트래픽 조회 실패: Expecting value: line 1 column 1 (char 0)
응답 원문: <?xml version="1.0" encoding="UTF-8"?>
<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/">
    <soapenv:Body>
        <soapenv:Fault>
            <faultcode>soapenv:Server</faultcode>
            <faultstring>Policy Falsified</faultstring>
            <faultactor>https://apis.data.go.kr/B552584/UserSportSvc/getSvkceyDalyStats?serviceKey=tN3HCezG7xu3AiGkV49yDcBplKq0pe9MUSui8ypmFt76fxJv2ckawdGI4brDFT25uhA%2FISsxlRP7Mv5mnkIKKQ%3D%3D&amp;searchDate=2025-07-02&amp;returnType=json&amp;numOfRows=100&amp;pageNo=1</faultactor>
            <detail>
                <l7:policyResult
                    status="Service Not Found.  The request may have been sent to an invalid URL, or intended for an unsupported operation." xmlns:l7="http://www.layer7tech.com/ws/policy/fault"/>
            </detail>
        </soapenv:Fault>
    </soapenv:Body>
</soapenv:Envelope>



In [ ]:
string_tmp = """⚠️ 인계동 4페이지에 데이터 없음
✅ 인계동 데이터 수집 완료 (2183개)
⚠️ 광교동 4페이지에 데이터 없음
✅ 광교동 데이터 수집 완료 (2183개)
⚠️ 영통동 4페이지에 데이터 없음
✅ 영통동 데이터 수집 완료 (2183개)
⚠️ 천천동 4페이지에 데이터 없음
✅ 천천동 데이터 수집 완료 (2182개)
⚠️ 경수대로(동수원) 4페이지에 데이터 없음
✅ 경수대로(동수원) 데이터 수집 완료 (2183개)
⚠️ 고색동 4페이지에 데이터 없음
✅ 고색동 데이터 수집 완료 (2183개)
⚠️ 호매실 4페이지에 데이터 없음
✅ 호매실 데이터 수집 완료 (2182개)
⚠️ 대왕판교로(백현동) 4페이지에 데이터 없음
✅ 대왕판교로(백현동) 데이터 수집 완료 (2183개)
⚠️ 단대동 4페이지에 데이터 없음
✅ 단대동 데이터 수집 완료 (2183개)
⚠️ 정자동 4페이지에 데이터 없음
✅ 정자동 데이터 수집 완료 (2183개)
⚠️ 수내동 4페이지에 데이터 없음
✅ 수내동 데이터 수집 완료 (2183개)
⚠️ 성남대로(모란역) 4페이지에 데이터 없음
✅ 성남대로(모란역) 데이터 수집 완료 (2111개)
⚠️ 복정동 4페이지에 데이터 없음
✅ 복정동 데이터 수집 완료 (2183개)
⚠️ 운중동 4페이지에 데이터 없음
✅ 운중동 데이터 수집 완료 (2183개)
⚠️ 상대원동 4페이지에 데이터 없음
✅ 상대원동 데이터 수집 완료 (2176개)
⚠️ 의정부동 4페이지에 데이터 없음
✅ 의정부동 데이터 수집 완료 (2183개)
⚠️ 의정부1동 4페이지에 데이터 없음
✅ 의정부1동 데이터 수집 완료 (2182개)
⚠️ 송산3동 4페이지에 데이터 없음
✅ 송산3동 데이터 수집 완료 (2183개)
⚠️ 안양8동 4페이지에 데이터 없음
✅ 안양8동 데이터 수집 완료 (2118개)
⚠️ 부림동 4페이지에 데이터 없음
✅ 부림동 데이터 수집 완료 (2183개)
⚠️ 호계3동 4페이지에 데이터 없음
✅ 호계3동 데이터 수집 완료 (2183개)
⚠️ 안양2동 4페이지에 데이터 없음
✅ 안양2동 데이터 수집 완료 (2180개)
⚠️ 철산동 4페이지에 데이터 없음
✅ 철산동 데이터 수집 완료 (2183개)
⚠️ 소하동 4페이지에 데이터 없음
✅ 소하동 데이터 수집 완료 (2183개)
⚠️ 고잔동 4페이지에 데이터 없음
✅ 고잔동 데이터 수집 완료 (2183개)
⚠️ 초지동 4페이지에 데이터 없음
✅ 초지동 데이터 수집 완료 (2183개)
⚠️ 본오동 4페이지에 데이터 없음
✅ 본오동 데이터 수집 완료 (2183개)
⚠️ 원곡동 4페이지에 데이터 없음
✅ 원곡동 데이터 수집 완료 (2181개)
⚠️ 부곡동1 4페이지에 데이터 없음
✅ 부곡동1 데이터 수집 완료 (2183개)
"""

string_tmp.count("✅")

29

In [ ]:
# 167개의 측정소 모두에서 3MONTH데이터를 가져왔나 확인하기

# 1. 기준 측정소 리스트 불러오기
station_info_df = pd.read_csv('/content/drive/MyDrive/graduation_project/station_info.csv')
standard_stations = set(station_info_df['stationName'].dropna().unique())

# 2. 측정 데이터 CSV 파일 리스트
data_files = [
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_070305.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_072258.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_073204.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_073915.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_095309.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_100842.csv'
]

# 3. 측정 데이터에서 측정소 이름 수집
observed_stations = set()

for file in data_files:
    df = pd.read_csv(file)
    observed_stations.update(df['stationName'].dropna().unique())

# 4. 비교
missing_stations = standard_stations - observed_stations
extra_stations = observed_stations - standard_stations

print(f"총 기준 측정소 수: {len(standard_stations)}")
print(f"총 관측된 측정소 수: {len(observed_stations)}\n")

if not missing_stations and not extra_stations:
    print("✅ 모든 167개 기준 측정소가 데이터에 포함되어 있습니다.")
else:
    print("❌ 일부 누락 또는 추가된 측정소가 있습니다.\n")

    if missing_stations:
        print(f"❌ 누락된 측정소 ({len(missing_stations)}개):")
        print(sorted(missing_stations))

    if extra_stations:
        print(f"⚠️ 기준 외 추가 측정소 ({len(extra_stations)}개):")
        print(sorted(extra_stations))

총 기준 측정소 수: 167
총 관측된 측정소 수: 167

✅ 모든 167개 기준 측정소가 데이터에 포함되어 있습니다.
